In [1]:
import base64
import based58
import sys
import json
import os
import time
from pathlib import Path

import solana.rpc.api
from solders.signature import Signature
from solana.exceptions import SolanaRpcException
from solders.rpc.responses import GetTransactionResp

from src.tx_decoder import TransactionDecoder


%load_ext autoreload
%autoreload 2
sys.path.append('..')


ppk = "MFv2hWf31Z9kbCa1snEPYctwafyhdvnV7FZnsebVacA"
idl_path = '../marginfi_idl2.json'


def get_tx(tx_signature):
    try:
        transaction = solana_client.get_transaction(
            tx_signature.signature, 
#             Signature.from_string(
#                 "4if5hzfKbJSxbA13ReBca2sY6kbAias7FseHG1QdUCHqSjhG4eNbGoz96TbBSNmCGoSQfKWx5gMtDVgxNb1iUV2M"
#             ),
            'jsonParsed', 
            max_supported_transaction_version=0
        )
        return transaction
    except SolanaRpcException as e:
        print(f"SolanaRpcException while fetching {str(tx_signature.signature)}")
        time.sleep(0.5)
        return get_tx(tx_signature)

    
def tx_printout(tx_parsed):

    
    print('===========logs============')
    for log, value in tx_parsed['logs'].items():
        print(f"{log} \n")
        print(f'      Parsed: {value} \n')
    
    print('=========relevant instructions=========')
    for program_id, instruction_data in tx_parsed['instructons']:
        print(f"Program ID: {program_id} \n   {instruction_data}")
    
    print('============Account Balances==========')
    for i in tx_parsed['account_balances']:
        print(i)
        
    print('============Token Balances==========')
    for i in tx_parsed['token_balances']:
        print(f"{i} \n")       
        

## Parse transactions from oldest to newest
- limited to one signer/ one mf account.


In [3]:
solana_client = solana.rpc.api.Client(os.getenv("RPC_URL"))

tx_decoder = TransactionDecoder(
    idl_path,  # mf ild
    solana.rpc.api.Pubkey.from_string(ppk)  # margin fi
)

In [4]:
signer = solana.rpc.api.Pubkey.from_string("DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc")

In [4]:
signer_tx_sinatures = solana_client.get_signatures_for_address(
    signer,
    limit = 1000
)
print(len(signer_tx_sinatures.value))

50


In [5]:
# collect signer related txs
signer_txs = list()

for tx in signer_tx_sinatures.value:
    signature = tx.signature
    transaction = get_tx(tx)
    signer_txs.append(transaction)

SolanaRpcException while fetching 5Pr4fc1sjFW9Vr3Z2MpWgMnZmHBLGQuvq6aHAveattJA1QSwuscYebmbzj3nBDVuh8v4bhzypgT8XJLgkLvdpbw5
SolanaRpcException while fetching 3fVZ7yhqeWzA17BrA88zbna5w98bBhiER49Ke8HhmKSrRz3Mxok5PgM69k7g2qBooLr85RrV2G4HEqztEz63yjFr
SolanaRpcException while fetching 3SXSLjSH1J8soDroS4noiVNFKqoWcdsUznuQpPNRbq5URCoqh8bckLWYPdFaJg5cXTTr74DLWx6w4wvfq43REHZ1


In [122]:
# print(f"Events collected: {len(tx_decoder.events)}")

# for tx in signer_txs:
#     tx_decoder.decode(tx.value.transaction, tx.value.slot)
    
# print(f"Events collected: {len(tx_decoder.events)}")    

Events collected: 0
Events collected: 13


In [7]:
# collect signatures related to marginfi account:
mf_account = solana.rpc.api.Pubkey.from_string("3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1")
mfacc_tx_sinatures = solana_client.get_signatures_for_address(
    mf_account,
    limit = 1000
)
print(f"Transactions to collect: {len(mfacc_tx_sinatures.value)}")
# collect mf account related txs
mfacc_txs = list()

for idx, tx in enumerate(mfacc_tx_sinatures.value):
    print(idx+1)
    signature = tx.signature
    transaction = get_tx(tx)
    mfacc_txs.append(transaction)
print(f"Collected: {len(mfacc_txs)}")

Transactions to collect: 429
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
SolanaRpcException while fetching 61BWQbDF3rUNbLP26X9SdsoWWSwLhTUnaoHw6AEoZU5iNseVk6HLhtXMZTMgPozZSyu7hi3ieD5KzC6Gbcj9QvVh
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
SolanaRpcException while fetching 4BysNFNPtgtsNHV6s4dDEidSWjihtgDjsoPXfDCDGu8hW5siBBAdTGFAjptMa9UJeqGUWMmZ2SHNnhn3kKmJ29d5
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
SolanaRpcException while fetching 43JTngTjUT7VmqcuPSZu1ZsCDFvpmnrez3gKwzLDVSk9DcWFgUCf6FzmAFAjZwKHjThpXhHWvkPbvkkbMvUtJ6iH
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
17

In [16]:
def dump_json_to_file(json_data, file_path):
    with open(file_path, 'w') as file:
        json.dump(json_data, file, indent=4)

def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


# path = './mf_txs/'
# for num, tx in enumerate(mfacc_txs):
#     dump_json_to_file(tx.to_json(), os.path.join(path, f"{'{:03d}'.format(num)}.json"))

## GET data ffrom local storage:

In [25]:
# mfacc_txs_2 = []
# for file_name in sorted(os.listdir('./mf_txs/')):
#     tx_json = load_json_from_file(os.path.join('./mf_txs/', file_name))
#     tx = GetTransactionResp.from_json(tx_json)
#     mfacc_txs_2.append(tx)
# print(len(mfacc_txs_2))

429


In [27]:
print(f"Events collected: {len(tx_decoder.events)}")

for tx in mfacc_txs:
    tx_decoder.decode(tx.value.transaction, tx.value.slot)
    
print(f"Events collected: {len(tx_decoder.events)}") 

Events collected: 0
Events collected: 527


In [29]:
events = sorted(tx_decoder.events, key=lambda x: x[0])

for k, i in enumerate(events):
    print(f"{k}   {i[0]} ---- {i[1]}")
    print(i[2].name)
    print(i[2].data)
    print(i[2].name)
    print('===============')

0   235690176 ---- 2nSZ3J63grj5qisbFtCcDr6K46hLmzcL4WtVKVYLYofDT2C6QK1j69AZxChCUs8EDDvzbZhuMDzBjY7fdLgkUSoG
MarginfiAccountCreateEvent
MarginfiAccountCreateEvent(header=AccountEventHeader(signer=Pubkey(
    DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc,
), marginfi_account=Pubkey(
    3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1,
), marginfi_account_authority=Pubkey(
    DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc,
), marginfi_group=Pubkey(
    4qp6Fx6tnZkY5Wropq9wUYgtFxXKwE6viZxFHg3rdAG8,
)))
MarginfiAccountCreateEvent
1   235690197 ---- 2WMjce2dMe2Lm7AXpQaAMrT9qtEfveTeNiSFZqXRkELmT8QhVGcmu8ir7VHcd63BBdaW9GUgqyBCJHubfNNWkynL
LendingPoolBankAccrueInterestEvent
LendingPoolBankAccrueInterestEvent(header=GroupEventHeader(signer=None, marginfi_group=Pubkey(
    4qp6Fx6tnZkY5Wropq9wUYgtFxXKwE6viZxFHg3rdAG8,
)), bank=Pubkey(
    2s37akK2eyBbp8DZgCm7RtsaEz8eJP3Nxd4urLHQv7yB,
), mint=Pubkey(
    EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,
), delta=17, fees_collected=0.0, insurance_collected=

In [158]:
bank = {
    'address': '2s37akK2eyBbp8DZgCm7RtsaEz8eJP3Nxd4urLHQv7yB',
    'amounts': dict()
}
mg_account = {
    'address': '3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1',
    'amounts': dict(),
    'debt': dict()
}
deposits_cnt = 0
borrows_cnt = 0


for n, (_, _, event) in enumerate(events):
    # LIQUIDATION
    if event.name == 'LendingAccountLiquidateEvent':
        if mg_account['address'] == str(event.data.liquidatee_marginfi_account):
            print(f"{n} SUCCESSFUL LIQUIDATION")
    
        else:
            print(f"{n} different MARGINFI accountin event {event.name}: {str(event.data.header.marginfi_account)}")
            
    # DEPOSIT
    if event.name == 'LendingAccountDepositEvent':
        if mg_account['address'] == str(event.data.header.marginfi_account):
            if str(event.data.mint) not in mg_account['amounts']:
                mg_account['amounts'][str(event.data.mint)] = 0
            mg_account['amounts'][str(event.data.mint)] += event.data.amount
            deposits_cnt += 1
                
        else:
            print(f"{n} different MARGINFI accountin event {event.name}: {str(event.data.header.marginfi_account)}")
  
    # BORROW
    if event.name == 'LendingAccountBorrowEvent':
        if mg_account['address'] == str(event.data.header.marginfi_account):
            if str(event.data.mint) not in mg_account['debt']:
                mg_account['debt'][str(event.data.mint)] = 0
            mg_account['debt'][str(event.data.mint)] += event.data.amount
            borrows_cnt += 1
                
        else:
            print(f"different MARGINFI accountin event {event.name}: {str(event.data.header.marginfi_account)}")
#     if event.name not in []
        
#         if bank['address'] == str(event.data.bank):
            
#         else:
#             print(f"different bank in event {event.name}: {str(event.data.bank)}")
print(f"# of deposits: {deposits_cnt}, \n# of borrows: {borrows_cnt}")
mg_account

14 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
19 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
22 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
25 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
27 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
30 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
33 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
46 SUCCESSFUL LIQUIDATION
55 SUCCESSFUL LIQUIDATION
106 SUCCESSFUL LIQUIDATION
146 different MARGINFI accountin event LendingAccountDepositEvent: L1QormaHsNEEhXcxn4scKGQVAwjuSznhLSUKKZh9UPf
147 SUCCESSFUL LIQUIDATION
150 different

{'address': '3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1',
 'amounts': {'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v': 1005668538,
  'So11111111111111111111111111111111111111112': 4389740686},
 'debt': {'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 7500000000}}

In [17]:
transaction.value.slot


235686490

## 'DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc'

mf account = 3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1

In [51]:
# 'DAhb626CWFq7GwmPPexXBU3jXLHf8MK478tNje81trPc'

for i, (ptx, tr_raw) in enumerate(zip(parsed_txs, transactions)):
    block = tr_raw.value.slot
    print(f'============={i} =======')
    logs = ptx['logs']
    for log, value in logs.items():
        print(f"{log} \n")
        print(f'      Parsed: {value} \n')
#     print('===instructions===')
#     for program_id, instruction_data in ptx['instructons']:
#         print(f"Program ID: {program_id} \n   {instruction_data}")
    

=============0 =======
Program 11111111111111111111111111111111 invoke [1] 

      Parsed: Foreign program/Nothing to parse 

Program 11111111111111111111111111111111 success 

      Parsed: Foreign program/Nothing to parse 

=============1 =======
Program ComputeBudget111111111111111111111111111111 invoke [1] 

      Parsed: Foreign program/Nothing to parse 

Program ComputeBudget111111111111111111111111111111 success 

      Parsed: Foreign program/Nothing to parse 

Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL invoke [1] 

      Parsed: Foreign program/Nothing to parse 

Program log: CreateIdempotent 

      Parsed: Foreign program/Nothing to parse 

Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2] 

      Parsed: Foreign program/Nothing to parse 

Program log: Instruction: GetAccountDataSize 

      Parsed: Foreign program/Nothing to parse 

Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 1569 of 1394595 compute units 

      Parsed: Foreign progr

get data for margin fi account:

In [53]:
res = solana_client.get_signatures_for_address(
    solana.rpc.api.Pubkey.from_string(
        '3pUsToJqWgT8j6zXrC5fHnD3YX987JbpV8EH2SzxV9k1'
    ), # Address 
    limit = 1000,
    before =  None
)